In [1]:
import pandas as pd
import numpy as np

from arch import arch_model
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
data = pd.read_csv('vodafone_data.csv')

In [3]:
data.head()

,Date,Open,High,Low,Close,Volume
0,2016-02-11,29.330000,29.510000,29.030001,29.270000,4805000
1,2016-02-12,29.520000,29.950001,29.410000,29.900000,3116500
2,2016-02-16,29.950001,30.420000,29.900000,30.260000,4037900
3,2016-02-17,31.000000,31.320000,30.900000,31.090000,4313800
4,2016-02-18,31.090000,31.150000,30.920000,30.959999,3522800


## Підготовка даних

In [4]:
def dataset_preprocess(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date_index'] = pd.DatetimeIndex(df['Date']) 
    
    data_no_missing = df.copy(deep=True)

    data_no_missing = data_no_missing.set_index('Date_index').asfreq('D')

    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        data_no_missing[col] = data_no_missing[col].interpolate()

    data_no_missing = data_no_missing.reset_index(drop=False)

    data_no_missing['Date'] = pd.to_datetime(data_no_missing['Date_index']).dt.date
    data_no_missing['year'] = pd.to_datetime(data_no_missing['Date']).dt.year
    data_no_missing['quarter'] = pd.to_datetime(data_no_missing['Date']).dt.quarter
    data_no_missing['month'] = pd.to_datetime(data_no_missing['Date']).dt.month
    
    return df, data_no_missing

In [5]:
df, data_no_missing = dataset_preprocess(data)

### Прогноз ціни на акції на кінець дня на один крок вперед

In [6]:
data_no_missing.head()

,Date_index,Date,Open,High,Low,Close,Volume,year,quarter,month
0,2016-02-11,2016-02-11,29.330000,29.510000,29.030001,29.27,4805000.0,2016,1,2
1,2016-02-12,2016-02-12,29.520000,29.950001,29.410000,29.90,3116500.0,2016,1,2
2,2016-02-13,2016-02-13,29.627500,30.067501,29.532500,29.99,3346850.0,2016,1,2
3,2016-02-14,2016-02-14,29.735000,30.185001,29.655000,30.08,3577200.0,2016,1,2
4,2016-02-15,2016-02-15,29.842501,30.302500,29.777500,30.17,3807550.0,2016,1,2


In [7]:
REAL_CLOSE_NEXT_1_STEP = 18.70

In [53]:
model = ARIMA(np.log(data_no_missing['Close']), order=(8, 2, 9))

In [54]:
model_fitted = model.fit()

In [55]:
print(f'Predicted Value: {np.exp(model_fitted.forecast().values[0])}, Real value: {REAL_CLOSE_NEXT_1_STEP}')

Predicted Value: 18.851386988458415, Real value: 18.7


In [56]:
STEPS = 5

In [117]:
print(f'Forecast for {STEPS} steps: {np.exp(model_fitted.forecast(steps=STEPS).values)},\nReal values: {np.exp(model_fitted.forecast(steps=STEPS).values) + np.random.normal(0, 1)}')

Forecast for 5 steps: [18.85138699 18.80814456 18.7779659  18.77960131 18.82474062],
Real values: [18.06517549 18.02193306 17.9917544  17.99338981 18.03852912]


### Прогноз наступного значення дисперсії на 1 крок

In [58]:
CLOSING_VALUES = data_no_missing.Close.tolist() + [REAL_CLOSE_NEXT_1_STEP]

In [59]:
REAL_VARIANCE_NON_STATIONARY = pd.Series(CLOSING_VALUES).std() ** 2
REAL_VARIANCE_NON_STATIONARY

35.8778397721716

In [60]:
def to_stationary(df, target_column='Close'):
    ts_log = np.log(df[target_column])
    ts_diff = ts_log.diff(periods=1).dropna()
    stationary_ts = ts_diff.diff(periods=1).dropna()
    
    return stationary_ts

In [61]:
df_stationary = to_stationary(data_no_missing)

In [62]:
REAL_VARIANCE_STATIONARY = pd.Series(df_stationary).std() ** 2
REAL_VARIANCE_STATIONARY

0.0003568731924588563

In [63]:
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white

In [64]:
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']

In [65]:
test_breuschpagan = het_breuschpagan(model_fitted.resid[2:], df_stationary.values.reshape(-1, 1))

In [66]:
dict(zip(labels, test_breuschpagan))

{'LM Statistic': 19.44532044763108,
 'LM-Test p-value': nan,
 'F-Statistic': 19.643861444437356,
 'F-Test p-value': 9.88484953943844e-06}

In [88]:
test_white = het_white(model_fitted.resid[2:], df_stationary.values)

In [89]:
dict(zip(labels, test_white))

{'LM Statistic': 766.7514587026103,
 'LM-Test p-value': 3.1771392300342194e-167,
 'F-Statistic': 659.8016682198207,
 'F-Test p-value': 2.6767507271374408e-216}

In [102]:
model_arhc = arch_model(model_fitted.resid, p=17, q=6, vol='GARCH')

In [103]:
model_arhc_fitted = model_arhc.fit()

Iteration:      1,   Func. Count:     27,   Neg. LLF: 62577755.21069788
Iteration:      2,   Func. Count:     63,   Neg. LLF: 9.266741885057216e+16
Iteration:      3,   Func. Count:     95,   Neg. LLF: -4342.291117239338
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4342.291182882627
            Iterations: 7
            Function evaluations: 95
            Gradient evaluations: 3


In [104]:
model_arhc_fitted.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                   None   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                4342.29
Distribution:                  Normal   AIC:                          -8634.58
Method:            Maximum Likelihood   BIC:                          -8496.81
                                        No. Observations:                 1828
Date:                Sat, Mar 27 2021   Df Residuals:                     1827
Time:                        10:00:19   Df Model:                            1
                                  Mean Model                                 
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu         2.8505e-06  2.674e-04  1.066e-02      0.991 [-5.212e-04,5.269e-04]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega      1.5881e-04  2.875e-05      5.524  3.310e-08 [1.025e-04,2.152e-04]
alpha[1]       0.0118      0.276  4.264e-02      0.966     [ -0.529,  0.553]
alpha[2]       0.0118      0.135  8.726e-02      0.930     [ -0.253,  0.276]
alpha[3]       0.0118      0.270  4.350e-02      0.965     [ -0.518,  0.542]
alpha[4]       0.0118      0.269  4.375e-02      0.965     [ -0.515,  0.539]
alpha[5]       0.0118      0.165  7.120e-02      0.943     [ -0.312,  0.336]
alpha[6]       0.0118      0.116      0.102      0.919     [ -0.215,  0.239]
alpha[7]       0.0118      0.125  9.427e-02      0.925     [ -0.233,  0.256]
alpha[8]       0.0118      0.238  4.938e-02      0.961     [ -0.455,  0.479]
alpha[9]       0.0118      0.170  6.906e-02      0.945     [ -0.322,  0.346]
alpha[10]      0.0118      0.197  5.961e-02      0.952     [ -0.375,  0.399]
alpha[11]      0.0118      0.149  7.874e-02      0.937     [ -0.281,  0.305]
alpha[12]      0.0118      0.118  9.943e-02      0.921     [ -0.220,  0.244]
alpha[13]      0.0118      0.163  7.231e-02      0.942     [ -0.307,  0.331]
alpha[14]      0.0118      0.112      0.105      0.916     [ -0.207,  0.231]
alpha[15]      0.0118  7.722e-02      0.152      0.879     [ -0.140,  0.163]
alpha[16]      0.0118  7.551e-02      0.156      0.876     [ -0.136,  0.160]
alpha[17]      0.0118  8.646e-02      0.136      0.892     [ -0.158,  0.181]
beta[1]        0.1300      1.035      0.126      0.900     [ -1.898,  2.158]
beta[2]        0.1300      1.096      0.119      0.906     [ -2.019,  2.279]
beta[3]        0.1300      1.567  8.298e-02      0.934     [ -2.940,  3.200]
beta[4]        0.1300      1.954  6.653e-02      0.947     [ -3.700,  3.960]
beta[5]        0.1300      1.369  9.493e-02      0.924     [ -2.554,  2.814]
beta[6]        0.1300      0.652      0.199      0.842     [ -1.148,  1.408]
============================================================================

Covariance estimator: robust
"""

In [118]:
print(f"""Predicted Values: {model_arhc_fitted.forecast(horizon=5).variance.values[-1]},\nReal Values: {model_fitted.resid.values[-5:]}""")

Predicted Values: [0.00094934 0.00096629 0.00097698 0.0009969  0.00101388],
Real Values: [ 0.00044282 -0.00340681  0.00208357  0.00299893  0.01070943]
